## Обучение пайплайна (шаг 1)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


### Замена значений

In [3]:
#Замена пустых строк
df['TotalCharges'].fillna(0, inplace=True)
#Замена строк на числа
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
med = df['TotalCharges'].median()
mean = df['TotalCharges'].mean()
std = df['TotalCharges'].std()
print(f'median - {mean}')
print(f'mean - {med}')
print(f'std - {std}')

median - 2283.3004408418697
mean - 1397.475
std - 2266.7713618831453


In [4]:
df['PhoneService'] = df['PhoneService'].replace({'Yes': 1, 'No': 0})

In [5]:
df.loc[df['TotalCharges'] ==' ', 'TotalCharges'] = np.nan
df['TotalCharges'] = df['TotalCharges'].astype(np.float32)
df['TotalCharges'].fillna(0, inplace=True)
med = df['TotalCharges'].median()
mean = df['TotalCharges'].mean()
std = df['TotalCharges'].std()
print(f'median - {mean}')
print(f'mean - {med}')
print(f'std - {std}')

median - 2279.732177734375
mean - 1394.550048828125
std - 2266.79443359375


In [6]:
# Обработка целевой переменной
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

In [7]:
df['gender'] = df['gender'].replace({'Female': 0, 'Male': 1, 'No internet service': 0})
df['StreamingMovies'] = df['StreamingMovies'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
df['StreamingTV'] = df['StreamingTV'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})
df['TechSupport'] = df['TechSupport'].replace({'Yes': 1, 'No': 0, 'No internet service': 0})

In [8]:
df["PhoneService"].fillna(0, inplace=True)

In [9]:
# Оставляем только выбранные признаки
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df = pd.DataFrame(df, columns=columns)
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,0,1,0,29.850000,0,0,0,0
1,1,34,1,1889.500000,0,0,0,0
2,1,2,1,108.150002,0,0,0,1


### Подготовка выборки

In [10]:
# Разделим датасет на тренировочную и тестовую выборку
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X = pd.DataFrame(df, columns=features)
X_train, X_test, y_train, y_test = train_test_split(X, df[target], test_size=0.3, random_state=42)

In [11]:
X_test.to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

### Сборка пайплайна

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

In [13]:
# Нормирование признаков
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

In [14]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

In [15]:
# Объединение всех признаков
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['gender','PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [16]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ])
feature_processing = Pipeline([('feats', feats)])

### Обучение пайплайна

In [17]:
# Для обучения используем модель RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

In [18]:
# Делаем прогноз
predict = pipeline.predict_proba(X_test)[0][1]
predict

0.72

In [19]:
# Делаем предсказание на одной строке данных (как при вводе в api)
pipeline.predict_proba(pd.DataFrame({'gender':0, 'tenure':[1], 
                                     'PhoneService':[0],'TotalCharges':[24.7999], 
                                     'StreamingMovies':[0], 'StreamingTV':[0], 'TechSupport':[0]}))

array([[0.28, 0.72]])

In [20]:
predict = pipeline.predict_proba(X_test)[: ,1]

### Метрики качества

In [21]:
from sklearn.metrics import roc_auc_score, log_loss

print(f'roc_auc_score = {np.round(roc_auc_score(y_test, predict), 3)}')
print(f'log_loss = {np.round(log_loss(y_test, predict), 3)}')

roc_auc_score = 0.793
log_loss = 0.705


### Сохранение пайплайна

In [22]:
import dill
with open("model_curs.dill", "wb") as f:
    dill.dump(pipeline, f)

Признаки: 
1. gender - пол (0, 1) (женский, мужской)
2. tenure - устройств во владении, тип - целое число
3. PhoneService - телефонное обслуживание (0, 1)
4. TotalCharges - суммарная стоимость, тип число
5. StreamingMovies - просмотр фильмлв (0, 1)
6. StreamingTV - просмотр ТВ (0, 1)
7. TechSupport - техническая поддержка (0, 1)

Целевая переменная: Churn  